# 필수 개인 과제
- LLM 과 RAG를 활용하여 AI 챗봇을 구현해보자.

### 필요한 라이브러리들을 불러온다.

In [49]:
import os   # 운영체제와 상호작용하게 해주는 표준 라이브러리
from dotenv import load_dotenv  # .env 파일에서 환경 변수를 불러올 수 있게 해주는 라이브러리

from langchain_openai import ChatOpenAI # LangChain 라이브러리에서 OpenAI의 챗봇 모델을 사용하기 위한 코드. 즉, OpenAI의 챗 모델을 사용하여 언어 모델을 초기화하기 위한 코드
from langchain_core.messages import HumanMessage    # LangChain 라이브러리에서 HumanMessage 클래스를 임포트하는 코드

from langchain_community.document_loaders import PyPDFLoader  # LangChain 라이브러리에서 PDF 파일을 로드하는 데 사용하는 코드

from langchain.text_splitter import CharacterTextSplitter   # 텍스트를 여러 덩어리로 분할하는 기능
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 긴 텍스트를 의미가 끊기지 않도록 일정한 길이의 조각으로 나누는 역할

from langchain_openai import OpenAIEmbeddings   # langchain_openai 라이브러리에서 OpenAIEmbeddings 클래스를 불러온다.

import faiss    # 벡터 검색을 위한 라이브러리
from langchain_community.vectorstores import FAISS  # # langchain_community에서 FAISS 벡터스토어 클래스를 임포트한다.

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
# LangChain의 ChatPromptTemplate와 RunnablePassthrough 클래스를 임포트한다.

_**HumanMessage**_ 클래스는 LangChain의 메시지 처리 시스템에서 사용자 메시지를 나타내는 데 사용된다.   
즉, LangChain에서 인간의 메시지를 모델로 전달하는 데 필요한 형태로 포맷팅해준다.   
LangChain은 대화형 AI 시스템을 구축할 때 여러 종류의 메시지를 처리하는데, HumanMessage는 사람이 보낸 메시지를 모델에 전달할 때 사용된다.

_**PyPDFLoader**_ 는 PDF 파일을 로드하여 텍스트를 추출하는 클래스다.   
PyPDFLoader를 langchain.document_loaders에서 임포트하는 것이 더 이상 권장되지 않기 때문에  
**langchain_community.document_loaders**에서 임포트해야 한다.

_**FAISS**_ (Facebook AI Similarity Search)는 고속 유사도 검색을 위한 라이브러리다. 주로 벡터 데이터베이스에서 빠르게 유사한 항목을 검색하는 데 사용된다.   
_**langchain_community.vectorstores**_ 는 Langchain의 벡터스토어 모듈로, 다양한 벡터 데이터베이스에 데이터를 저장하고 검색할 수 있는 기능을 제공한다. 여기서는 FAISS 벡터스토어를 사용하여 문서들을 벡터화하고 저장하려고 하고 있다.

## 1. 사용환경 준비

In [50]:
# .env 파일 로드
load_dotenv()   # .env 파일을 로드하여 환경 변수들을 설정한다.

True

In [51]:
# .env 파일에서 api 키 가져오기
API_KEY = os.getenv('sparta_api_key')

_**os.getenv**_ 는 환경 변수에서 값을 가져오는 역할을 한다. 여기서는 api 키 값을 가져왔다.  
즉, .env 파일에 저장된 'sparta_api_key'라는 환경 변수의 값을 가져왔다.

In [52]:
# API 키가 잘 로드되었는지 확인
if API_KEY is None:
    raise ValueError("API key is missing from .env file")
# print(API_KEY)

_**print(API_KEY)**_ 를 통해 API_KEY를 직접 확인해볼 수도 있다.

In [53]:
# 환경 변수에 API 키 설정
os.environ['OPENAI_API_KEY'] = API_KEY

이렇게 설정하면 openai 라이브러리가 환경 변수를 사용한다.  
openai.api_key = API_KEY ==> 이건 옛날 버전이니 위의 코드를 사용하자.  
이제 OpenAI 라이브러리가 자동으로 환경 변수를 사용하여 API 키를 설정한다.  
ChatOpenAI 모델을 초기화 할 때 API 키를 별도로 전달할 필요가 없다.

```python
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")
```
이 코드를 통해 api key가 잘 작동하는 지 확인할 수도 있다.

## 2. 모델 로드하기

In [54]:
# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini") # ChatOpenAI 객체를 초기화하여 사용할 모델을 설정한다.

## 3. 문서 로드하기

In [55]:
# PyPDFLoader 인스턴스를 생성하고 PDF 파일을 로드할 준비를 한다. 파일의 경로 입력
loader = PyPDFLoader("./[2024 한권으로 ok 주식과 세금].pdf")

In [56]:
# PDF에서 텍스트 추출. 페이지 별 문서 로드
docs = loader.load()    # loader.load()를 호출하여 PDF 파일의 텍스트를 페이지별로 불러온다.

아래 코드를 통해 추출된 텍스트를 확인해볼 수 있다.
```python
for doc in docs:
    print(doc)
```

## 4. 문서 청크로 나누기

1. CharacterTextSplitter

In [57]:
text_splitter = CharacterTextSplitter(
    separator="\n\n",   # 두 개의 개행 문자를 구분자로 사용
    chunk_size=100, # 최대 100자씩 나눈다.
    chunk_overlap=10,   # 각 조각에 앞뒤로 5자의 중복을 포함한다.
    length_function=len,    # 길이를 계산할 때 문자 수(len)를 기준으로 한다.
    is_separator_regex=False,   # 구분자를 단순 문자열로 처리
)

splits = text_splitter.split_documents(docs)

print(splits[:10]) # 청킹된 내용 상위 10개 출력

[Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 2}, page_content='지난해 말 국내 상장법인 주식을 보유한 개인 투자자가 1,400만명을 넘어서는 등  \n국민들의 주식시장에 대한 관심이 크게 증가하였습니다.\n최근 일반 국민들의 주식투자에 대한 관심이 크게 증가했음에도 불구하고, 주식 투자  \n관련 세금문제 등 궁금한 사항에 대하여 도움을 줄 수 있는 안내책자 등은 시중에서  \n쉽게 찾아보기 어려운 게 현실입니다.\n이에 국세청에서는 주식 관련 각종 세금에 대한 납세자들의 이해를 높이고 납세의무  \n이행에 도움이 될 수 있도록 「주식과 세금」 책자를 처음으로 제작·발간하게 되었습니다.\n이번에 새롭게 출간하는 ‘주식과 세금’ 책자는 주식거래의 기초상식과 주식의 취득  \n부터 보유 및 처분시까지 단계별 세금문제를 총 76개의 문답형식으로 구성하는 한편, \n인포그래픽 등을 적극 활용하여 가독성을 제고하였으며, 구체적인 절세 꿀팁 및 자주 \n발생하는 실수 사례 등을 추가하여 활용성도 강화하였습니다.\n모쪼록, 이 책자가 주식등 관련 납세자들의 성실한 납세의무 이행에 기여할 수 있기를\n기대합니다.\n2024.  5\n국세청 자산과세국장\n머리말'), Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 3}, page_content='본 책자에 수록된 내용은 세법에서 규정하고 있는 내용을 알기 쉽게 요약하여 서술한\n것으로 모든 법령규정을 담고 있지는 않습니다.\n또한, 법령이 개정되거나 법령의 해석이 변경되어 본 책자의 내용과 다른 경우가 발생\n할 수 있으므로 실제 사안에 적용하는 경우 반드시 관련 법령과 해석 등을 충분히 확인\n하시기 바랍니다.\n본 책자는 발간일 현재 개정된 법령 등을 기준으로 작성되었습니다. 다만, 시행이  \n유예된 법령 등은 반영되어 있지 않습니다.\n본 책자에 표기된 

2. RecursiveCharacterTextSplitter

In [58]:
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, # 최대 100자씩 나눈다.
    chunk_overlap=10,   # 각 조각에 앞뒤로 5자의 중복을 포함한다.
    length_function=len,    # 길이를 계산할 때 문자 수(len)를 기준으로 한다.
    is_separator_regex=False,   # 구분자를 단순 문자열로 처리
)

splits_RCT = recursive_text_splitter.split_documents(docs)

print(splits_RCT[:10])  # 청킹된 내용 상위 10개 출력

[Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 2}, page_content='지난해 말 국내 상장법인 주식을 보유한 개인 투자자가 1,400만명을 넘어서는 등  \n국민들의 주식시장에 대한 관심이 크게 증가하였습니다.'), Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 2}, page_content='최근 일반 국민들의 주식투자에 대한 관심이 크게 증가했음에도 불구하고, 주식 투자  \n관련 세금문제 등 궁금한 사항에 대하여 도움을 줄 수 있는 안내책자 등은 시중에서'), Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 2}, page_content='쉽게 찾아보기 어려운 게 현실입니다.\n이에 국세청에서는 주식 관련 각종 세금에 대한 납세자들의 이해를 높이고 납세의무'), Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 2}, page_content='이행에 도움이 될 수 있도록 「주식과 세금」 책자를 처음으로 제작·발간하게 되었습니다.\n이번에 새롭게 출간하는 ‘주식과 세금’ 책자는 주식거래의 기초상식과 주식의 취득'), Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 2}, page_content='부터 보유 및 처분시까지 단계별 세금문제를 총 76개의 문답형식으로 구성하는 한편, \n인포그래픽 등을 적극 활용하여 가독성을 제고하였으며, 구체적인 절세 꿀팁 및 자주'), Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 2}, page_content='발생하는 실수 사례 등을 추가하여 활용성도 강화하였습니다.\n모쪼록

과제에서는 한 가지 청킹 방식만 사용하면 된다.   
나는 _**RecursiveCharacterTextSplitter**_ 청킹 방식을 선택했다.

## 5. 벡터 임베딩 생성
OpenAI 모델을 사용했기 때문에 OpenAIEmbeddings를 이용해 텍스트를 벡터로 변환할 벡터 임베딩을 생성했다.

In [59]:
# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

'text-embedding-ada-002' 모델을 사용하여 텍스트 데이터를 임베딩 벡터로 변환한다.   
이 모델은 텍스트 데이터를 고차원 벡터로 변환하는 데 사용된다.   
자연어 처리(NLP) 작업에서 텍스트를 벡터 공간으로 변환하여, 유사도 검색, 클러스터링, 분류 등 다양한 작업에 활용된다.

## 6. 벡터 스토어 생성

앞서 만든 벡터 임베딩과 청크된 문서를 활용하여 FAISS 벡터 스토어를 생성했다.

In [60]:
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

_**FAISS.from_documents**_ 는 splits_RCT라는 문서 목록을 받아들여, 각 문서를 임베딩 모델(embeddings)을 사용하여 벡터로 변환한 후, 이 벡터들을 FAISS 벡터스토어에 저장하는 역할을 한다.   
_**splits_RCT**_ 는 우리가 pdf 문서들을 chunk 단위로 나눈 데이터다. 이 데이터를 벡터로 변환하여 검색할 수 있게 documents에 저장한다.   
_**embedding**_ 에는 텍스트 임베딩 모델을 지정한다. 여기서는 OpenAI의 text-embedding-ada-002 모델을 사용하여 문서 벡터를 생성한다.

## 7. FAISS를 Retriever로 변환

RAG 체인에서 사용할 수 있도록 FAISS를 retriever로 변환하자.

In [61]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

_**RAG**_ (Retrieval-Augmented Generation) 체인에서 FAISS를 retriever로 변환하는 이유는 검색 기반 텍스트 생성을 효율적으로 수행하기 위해서다. RAG 체인에서는 **검색**과 **생성**의 두 단계가 중요한 역할을 한다. 이를 통해 모델은 주어진 질문에 대한 답변을 더 잘 생성할 수 있도록 외부 지식을 활용한다.

_**as_retriever()**_ 메서드는 vectorstore를 retriever로 변환하는 역할을 한다. 이 메서드는 벡터 데이터베이스에서 검색을 수행할 수 있도록 변환해 준다.  
_**search_type="similarity"**_ 에 의해 유사도 검색이 실행되고, _**{"k": 1}**_ 설정에 따라 가장 유사한 문서 1개만 검색해서 반환한다.

## 8. 프롬프트 템플릿 정의

프롬프트 템플릿을 정의하자.

In [62]:
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

_**system**_ 메시지를 통해 모델에게 주어진 문맥 내에서만 질문에 답하라는 지시를 내렸다.   
_**user**_ 메시지에는 실제 _**context**_ 와 _**question**_ 이 채워질 수 있도록 변수를 포함한 프롬프트 템플릿을 생성했다.   
따라서 {context}와 {question}은 나중에 동적으로 입력값에 따라 채워지게 된다.   
   
즉, 이 템플릿은 이후 사용자가 'context'와 'question'을 제공했을 때,   
"Context: <문맥 내용>\\n\\nQuestion: <질문 내용>" 형태로 메시지를 만들어내게 된다.